In [159]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pulp as plp
import networkx as nx
from ALB_instance_tools import *
from collections import namedtuple
import glob
import os
import string


In [160]:
def sum_prob(sequences):
    '''function for sanity checking that the probabilities sum to 1'''
    total = 0
    for seq in sequences:
        total += sequences[seq]['probability']
    return total

def make_consecutive_luxury_models_restricted_scenario_tree(n_takts, entry_probabilities, max_consecutive=3, luxury_models=['B']):
    """
    Creates a scenario tree for the given number of takts, instance and entry probabilities. 
    This scenario tree restricts the number of consective times a set of  model, the "luxury" models, can enter the line
    """
    # Create a directed graph
    G = nx.DiGraph()
    # Add the root node
    G.add_node('R', stage=0, scenario=0)
    #Create a list of final sequences
    final_sequences = {}
    def add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability=1, parent=0, sequence = [], current_stage=0, counter=[0], consecutive=0):
        if current_stage == n_takts:
            final_sequences[counter[0]] = {'sequence':sequence, 'probability': probability}
            counter[0] += 1
            return
        else:
            #Handle case where the model is the first one or is not the luxury model
            if len(sequence) == 0 or sequence[-1] not in  luxury_models:
                for model, prob in entry_probabilities.items():
                    new_sequence = sequence.copy()
                    new_sequence.append(model)
                    node_name = str(parent)+ str(model) 
                    graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                    graph.add_edge(parent, node_name, probability = probability)
                    add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1, consecutive=1)
            else:
                #Create a dictionary of models that excludes the previous model
                entry_probabilities_excluding_previous = entry_probabilities.copy()
                entry_probabilities_excluding_previous.pop(sequence[-1], None)
                #Handle case where the model is the same as the previous one
                if consecutive < max_consecutive:
                    new_sequence = sequence.copy()
                    model = new_sequence[-1]
                    prob = entry_probabilities[model]
                    new_sequence.append(model)
                    node_name = str(parent)+ str(model) 
                    graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                    graph.add_edge(parent, node_name, probability = probability)
                    add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1, consecutive=consecutive+1)
                else:
                    #If there are too many of this model, ignore it Change the entry probabilities of the other models proportional to their probability
                    total_prob = sum(entry_probabilities_excluding_previous.values())
                    for model, prob in entry_probabilities_excluding_previous.items():
                        entry_probabilities_excluding_previous[model] = prob/total_prob
                for model, prob in entry_probabilities_excluding_previous.items():
                    new_sequence = sequence.copy()
                    new_sequence.append(model)
                    node_name = str(parent)+ str(model) 
                    graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                    graph.add_edge(parent, node_name, probability = probability)
                    add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1, consecutive=1)
    add_nodes(n_takts, entry_probabilities, G, final_sequences, parent='R')
    return G, final_sequences

def make_consecutive_model_restricted_scenario_tree(n_takts, entry_probabilities, max_consecutive=3):
    """
    Creates a scenario tree for the given number of takts, instance and entry probabilities. 
    This scenario tree restricts the number of consective times a model can enter the line
    """
    # Create a directed graph
    G = nx.DiGraph()
    # Add the root node
    G.add_node('R', stage=0, scenario=0)
    #Create a list of final sequences
    final_sequences = {}
    def add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability=1, parent=0, sequence = [], current_stage=0, counter=[0], consecutive=0):
        if current_stage == n_takts:
            final_sequences[counter[0]] = {'sequence':sequence, 'probability': probability}
            counter[0] += 1
            return
        else:
            #Handle case where the model is the first one
            if len(sequence) == 0:
                for model, prob in entry_probabilities.items():
                    new_sequence = sequence.copy()
                    new_sequence.append(model)
                    node_name = str(parent)+ str(model) 
                    graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                    graph.add_edge(parent, node_name, probability = probability)
                    add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1, consecutive=1)
            else:
                #Create a dictionary of models that excludes the previous model
                entry_probabilities_excluding_previous = entry_probabilities.copy()
                entry_probabilities_excluding_previous.pop(sequence[-1], None)
                #Handle case where the model is the same as the previous one
                if consecutive < max_consecutive:
                    new_sequence = sequence.copy()
                    model = new_sequence[-1]
                    prob = entry_probabilities[model]
                    new_sequence.append(model)
                    node_name = str(parent)+ str(model) 
                    graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                    graph.add_edge(parent, node_name, probability = probability)
                    add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1, consecutive=consecutive+1)
                else:
                    #If there are too many of this model, ignore it Change the entry probabilities of the other models proportional to their probability
                    total_prob = sum(entry_probabilities_excluding_previous.values())
                    for model, prob in entry_probabilities_excluding_previous.items():
                        entry_probabilities_excluding_previous[model] = prob/total_prob
                for model, prob in entry_probabilities_excluding_previous.items():
                    new_sequence = sequence.copy()
                    new_sequence.append(model)
                    node_name = str(parent)+ str(model) 
                    graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                    graph.add_edge(parent, node_name, probability = probability)
                    add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1, consecutive=1)
    add_nodes(n_takts, entry_probabilities, G, final_sequences, parent='R')
    return G, final_sequences

def make_scenario_tree(n_takts, entry_probabilities):
    """
    Creates a scenario tree for the given number of takts, instance and entry probabilities.
    """
    # Create a directed graph
    G = nx.DiGraph()
    # Add the root node
    G.add_node('R', stage=0, scenario=0)
    #Create a list of final sequences
    final_sequences = {}
    def add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability=1, parent=0, sequence = [], current_stage=0, counter=[0]):
        if current_stage == n_takts:
            final_sequences[counter[0]] = {'sequence':sequence, 'probability': probability}
            counter[0] += 1
            return
        else:
            for model, prob in entry_probabilities.items():
                new_sequence = sequence.copy()
                new_sequence.append(model)
                node_name = str(parent)+ str(model) 
                graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                graph.add_edge(parent, node_name, probability = probability)
                add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1)
    add_nodes(n_takts, entry_probabilities, G, final_sequences, parent='R')
    return G, final_sequences
NO_takts = 5
scenario_tree_graph, final_sequences = make_scenario_tree(NO_takts, {'A':0.75, 'B':0.25})
restricted_graph, restricted_sequences = make_consecutive_luxury_models_restricted_scenario_tree(NO_takts, {'A':0.75, 'B':0.25}, max_consecutive=3, luxury_models=['A', 'B'])
restricted_total = sum_prob(restricted_sequences)
restricted_total

1.0

In [161]:
def check_scenarios(prod_sequence1,prod_sequence2,t):
    '''compares two production sequences up to time t and returns true if they are the same'''
    if prod_sequence1[:t+1] == prod_sequence2[:t+1]:
        return True
    else:
        return False


In [162]:
def add_non_anticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, num_stations):
    '''adds the non participation constraint for two scenarios w and w_prime'''
    #We go backwards in time, look for first time t where the sequences are the same
    for t in reversed(range(sequence_length)):
        if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], t):
            for j in range(t+1):
                model = prod_sequences[w]['sequence'][j]
                max_station = min(t-j+1, num_stations)    
                for s in range(max_station):
                    for o in range(model_instance[model]['num_tasks']):
                        prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                                f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')
            return


In [163]:
def old_stochastic_problem_linear_labour(model_instance, equipment_instance,no_tasks, NO_WORKERS, NO_STATIONS,takt_time, sequence_length, prod_sequences, worker_cost =100):
    print('Writing problem')
    print('Number of tasks:', no_tasks, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    c_se = equipment_instance['equipment_prices']
    R_oe = equipment_instance['equipment_matrix']
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (prod_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_wsoj = plp.LpVariable.dicts('x_wsoj', (prod_sequences.keys(), stations, range(no_tasks), range(sequence_length) ), lowBound=0, cat='Binary')
    #Defining LP problem
    prob = plp.LpProblem("stochastic_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                      [prod_sequences[w]['probability']/sequence_length * worker_cost* l * b_wtsl[w][t][s][l]
                         for w in prod_sequences.keys()
                           for t in range(0,sequence_length) 
                           for s in stations for l in workers]),
                "Total cost")
    #Constraints
    #Constraint 1 -- can only assign l number of workers to a station for a given scenario and stage
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
        #Constraint 2 all tasks must be assigned to a station
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
            #but this is more general
            for o in range(model_instance[model]['num_tasks']): 
                prob += (plp.lpSum([x_wsoj[w][s][o][j] for s in stations]) == 1, f'x_wsoj_{w}_s_{o}_{j}')
        #Constraint 3 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_wsoj[w][s][int(o)-1][j] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 4 -- tasks can only be assigned to a station with the correct equipment
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for s in stations:
                for o in range(model_instance[model]['num_tasks']):
                    prob += x_wsoj[w][s][o][j] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_wsoj_{w}_{s}_{o}_{j}'
        #Constraint 5 -- precedence constraints
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for (pred, suc) in model_instance[model]['precedence_relations']:
                prob += (plp.lpSum([ (s+1)  * x_wsoj[w][s][int(pred)-1][j] for s in stations])
                         <=  
                         plp.lpSum([ (s+1)  * x_wsoj[w][s][int(suc)-1][j] for s in stations]), 
                         f'task{pred} before task{suc} for model{model}, item {j} seq {w}' )
        #Constraint 6 -- non-anticipativity constraints
    for w in prod_sequences.keys():
        for w_prime in prod_sequences.keys():
            if w_prime > w:
                add_non_anticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, NO_STATIONS)
                # for j in reversed(range(sequence_length)):
                #     if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], j):
                #         model = prod_sequences[w]['sequence'][j]
                #         print('w', w, 'w_prime', w_prime)
                #         print('j', j, 'max_station', NO_STATIONS-j, 'model', model)
                #         for s in range(NO_STATIONS - j ):
                #             for o in range(model_instance[model]['num_tasks']):
                #                 prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                #                         f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')

                
    return prob

def old_model_dependent_eq_linear_labour(model_instance, equipment_instance,no_tasks, NO_WORKERS, NO_STATIONS,takt_time, sequence_length, prod_sequences, worker_cost =100, fixed_assignment = False):
    print('model instance', model_instance)
    print('Writing model dependent equivalent of problem')
    no_models = len(list(model_instance.keys()))
    print('number_of models', no_models,'Number of tasks:', no_tasks, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    c_se = equipment_instance['equipment_prices']
    R_oe = equipment_instance['equipment_matrix']
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (prod_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_soi = plp.LpVariable.dicts('x_soi', ( stations, range(no_tasks), model_instance.keys() ), lowBound=0, cat='Binary')
    #Defining LP problem
    prob = plp.LpProblem("model_dependent_eq_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                      [prod_sequences[w]['probability']/sequence_length * worker_cost* l * b_wtsl[w][t][s][l]
                         for w in prod_sequences.keys()
                           for t in range(0,sequence_length) 
                           for s in stations for l in workers]),
                "Total cost")
    #Constraints
    #constraints work over all scenarios
    # for w in final_sequences.keys():
        #Constraint 1 -- can only assign l number of workers to a station for a given scenario and stage
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
    #Constraint 2 all tasks must be assigned to a station
    for i, model in enumerate(model_instance.keys()):
        #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
        #but this is more general
        for o in range(model_instance[model]['num_tasks']): 
            prob += (plp.lpSum([x_soi[s][o][model] for s in stations]) == 1, f'x_soi_{s}_{o}_{model}')
        #Constraint 3 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_soi[s][int(o)-1][model] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 4 -- tasks can only be assigned to a station with the correct equipment
    for i, model in enumerate(model_instance.keys()):
        for s in stations:
            for o in range(model_instance[model]['num_tasks']):
                prob += x_soi[s][o][model] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_soj_{s}_{o}_{model}'
        #Constraint 5 -- precedence constraints
    for i, model in enumerate(model_instance.keys()):
        for (pred, suc) in model_instance[model]['precedence_relations']:
            prob += (plp.lpSum([ (s+1)  * x_soi[s][int(pred)-1][model] for s in stations])
                        <=  
                        plp.lpSum([ (s+1)  * x_soi[s][int(suc)-1][model] for s in stations]), 
                        f'task{pred} before task{suc} for model{model} ' )
    
    #Constraint 6 -- fixed task assignment (optional)
    if fixed_assignment:
        for i, model in enumerate(model_instance.keys()):
            for i_2, model_2 in enumerate(model_instance.keys()):
                if i != i_2:
                    for s in stations:
                        for o in range(model_instance[model]['num_tasks']):
                            prob += (x_soi[s][o][model] == x_soi[s][o][model_2], f'fixed_task_{s}_{o}_{model}_{model_2}')

    return prob

In [164]:
def stochastic_problem_linear_labour(model_instance, equipment_instance,no_tasks, 
                                     NO_WORKERS, NO_STATIONS,takt_time, sequence_length, 
                                     prod_sequences, worker_cost =100):
    print('Writing problem')
    print('Number of tasks:', no_tasks, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    c_se = equipment_instance['equipment_prices']
    R_oe = equipment_instance['equipment_matrix']
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (prod_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_wsoj = plp.LpVariable.dicts('x_wsoj', (prod_sequences.keys(), stations, range(no_tasks), range(sequence_length) ), lowBound=0, cat='Binary')
    Y_w = plp.LpVariable.dicts('Y_w', (prod_sequences.keys()), lowBound=0, cat='Integer')
    #Defining LP problem
    prob = plp.LpProblem("stochastic_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                      [prod_sequences[w]['probability']*Y_w[w]* worker_cost
                         for w in prod_sequences.keys()
                        ]),
                "Total cost")
    #Constraints
    #Constraint 1 -- Must hire Y workers if we use Y workers in a given takt
    for w in prod_sequences.keys():
        for t in takts:
            prob += (plp.lpSum([b_wtsl[w][t][s][l] for s in stations for l in workers]) <= Y_w[w], f'Y_w_{w}_{t}')
    #Constraint 2 -- can only assign l number of workers to a station for a given scenario and stage
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
        #Constraint 3 all tasks must be assigned to a station
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
            #but this is more general
            for o in range(model_instance[model]['num_tasks']): 
                prob += (plp.lpSum([x_wsoj[w][s][o][j] for s in stations]) == 1, f'x_wsoj_{w}_s_{o}_{j}')
        #Constraint 4 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_wsoj[w][s][int(o)-1][j] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 5 -- tasks can only be assigned to a station with the correct equipment
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for s in stations:
                for o in range(model_instance[model]['num_tasks']):
                    prob += x_wsoj[w][s][o][j] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_wsoj_{w}_{s}_{o}_{j}'
        #Constraint 6 -- precedence constraints
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for (pred, suc) in model_instance[model]['precedence_relations']:
                prob += (plp.lpSum([ (s+1)  * x_wsoj[w][s][int(pred)-1][j] for s in stations])
                         <=  
                         plp.lpSum([ (s+1)  * x_wsoj[w][s][int(suc)-1][j] for s in stations]), 
                         f'task{pred} before task{suc} for model{model}, item {j} seq {w}' )
        #Constraint 7 -- non-anticipativity constraints
    for w in prod_sequences.keys():
        for w_prime in prod_sequences.keys():
            if w_prime > w:
                add_non_anticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, NO_STATIONS)
                # for j in reversed(range(sequence_length)):
                #     if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], j):
                #         model = prod_sequences[w]['sequence'][j]
                #         print('w', w, 'w_prime', w_prime)
                #         print('j', j, 'max_station', NO_STATIONS-j, 'model', model)
                #         for s in range(NO_STATIONS - j ):
                #             for o in range(model_instance[model]['num_tasks']):
                #                 prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                #                         f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')

                
    return prob

def model_dependent_eq_linear_labour(model_instance, equipment_instance,no_tasks, NO_WORKERS, NO_STATIONS,takt_time, sequence_length, prod_sequences, worker_cost =100, fixed_assignment = False):
    print('model instance', model_instance)
    print('Writing model dependent equivalent of problem')
    no_models = len(list(model_instance.keys()))
    print('number_of models', no_models,'Number of tasks:', no_tasks, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    c_se = equipment_instance['equipment_prices']
    R_oe = equipment_instance['equipment_matrix']
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (prod_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_soi = plp.LpVariable.dicts('x_soi', ( stations, range(no_tasks), model_instance.keys() ), lowBound=0, cat='Binary')
    Y_w = plp.LpVariable.dicts('Y_w', (prod_sequences.keys()), lowBound=0, cat='Integer')

    #Defining LP problem
    prob = plp.LpProblem("model_dependent_eq_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                       [prod_sequences[w]['probability']*Y_w[w]* worker_cost
                         for w in prod_sequences.keys()
                        ]),
                "Total cost")
    #Constraints
    #Constraint 1 -- Must hire Y workers if we use Y workers in a given takt
    for w in prod_sequences.keys():
        for t in takts:
            prob += (plp.lpSum([b_wtsl[w][t][s][l] for s in stations for l in workers]) <= Y_w[w], f'Y_w_{w}_{t}')

    #Constraint 2 -- can only assign l number of workers to a station for a given scenario and stage
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
    #Constraint 3 all tasks must be assigned to a station
    for i, model in enumerate(model_instance.keys()):
        #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
        #but this is more general
        for o in range(model_instance[model]['num_tasks']): 
            prob += (plp.lpSum([x_soi[s][o][model] for s in stations]) == 1, f'x_soi_{s}_{o}_{model}')
        #Constraint 4 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_soi[s][int(o)-1][model] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 5 -- tasks can only be assigned to a station with the correct equipment
    for i, model in enumerate(model_instance.keys()):
        for s in stations:
            for o in range(model_instance[model]['num_tasks']):
                prob += x_soi[s][o][model] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_soj_{s}_{o}_{model}'
        #Constraint 6 -- precedence constraints
    for i, model in enumerate(model_instance.keys()):
        for (pred, suc) in model_instance[model]['precedence_relations']:
            prob += (plp.lpSum([ (s+1)  * x_soi[s][int(pred)-1][model] for s in stations])
                        <=  
                        plp.lpSum([ (s+1)  * x_soi[s][int(suc)-1][model] for s in stations]), 
                        f'task{pred} before task{suc} for model{model} ' )
    
    #Constraint 7 -- fixed task assignment (optional)
    if fixed_assignment:
        for i, model in enumerate(model_instance.keys()):
            for i_2, model_2 in enumerate(model_instance.keys()):
                if i != i_2:
                    for s in stations:
                        for o in range(model_instance[model]['num_tasks']):
                            prob += (x_soi[s][o][model] == x_soi[s][o][model_2], f'fixed_task_{s}_{o}_{model}_{model_2}')

    return prob
#prob = stochastic_problem_linear_labour(test_instances, equipment_instance[1],no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_takts, final_sequences, worker_cost=WORKER_COST)                                


In [165]:

def generate_report_stochastic(pulp_prob, sequences, instances, file_name):
    '''Shows task assignments for fixed and model dependent task assignment'''
    task_assignments = []
    labor_assignments = []
    labor_hire_assignments = []
    for v in pulp_prob.variables():
        if round(v.varValue) > 0:
            
            if 'x_wsoj' in v.name:
                sequence = int(v.name.split('_')[2])
                item = int(v.name.split('_')[5])
                model = sequences[sequence]['sequence'][item]
                #change the task number to match with the instances
                task = str(int(v.name.split('_')[4])+1)
                task_time = instances[model]['task_times'][task]
                assignment = {'scenario':v.name.split('_')[2], 'station': v.name.split('_')[3],'sequence_loc':item, 'model':model  , 'task': task, 'task_times': task_time}
                task_assignments.append(assignment)
            elif 'b_wtsl' in v.name:
                model = sequences[int(v.name.split('_')[2])]['sequence'][int(v.name.split('_')[4])]
                labor = {'scenario':v.name.split('_')[2], 'stage':v.name.split('_')[3], 
                         'station': v.name.split('_')[4], 'model':model, 'workers': int(v.name.split('_')[5]) }
                labor_assignments.append(labor)
            elif 'Y_w' in v.name:
                print(v.name.split('_')[2])
                labor_hire = {'scenario':v.name.split('_')[2], 'scenario_workers': int(v.value()) }
                labor_hire_assignments.append(labor_hire)
                print(v.name, v.varValue)

    #turns task_assignments into a dataframe
    task_assignments_df = pd.DataFrame(task_assignments)
    labor_assignments_df = pd.DataFrame(labor_assignments)
    labor_hire_df = pd.DataFrame(labor_hire_assignments)
    print('labor_hire_df', labor_hire_df)
    #concatenates the 'task' column in task_assignments_df if the 'station' and 'model' columns are the same
    task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()
    labor_assignments_df['sequence_loc'] = labor_assignments_df['stage'].astype(int) - labor_assignments_df['station'].astype(int)
    labor_assignments_df = labor_assignments_df[labor_assignments_df['sequence_loc'] >= 0]
    task_seq = task_assignments_df[['scenario','station', 'task','task_times', 'sequence_loc']]
    #merging labor and task sequence dataframes
    labor_task_seq = pd.merge(labor_assignments_df, task_seq, on=['scenario','station', 'sequence_loc'], how='left')
    labor_task_seq = pd.merge(labor_task_seq, labor_hire_df, on=['scenario'], how='left')
    task_assignments_df.to_csv(file_name + f'task_assignment.csv', sep=' ')
    labor_task_seq.to_csv(file_name + f'labor_assignment.csv', sep=' ')
    return task_assignments_df, labor_assignments_df

def generate_report_md(pulp_prob, sequences, instances, file_name):
    '''Shows task assignments for fixed and model dependent task assignment'''
    task_assignments = []
    labor_assignments = []
    labor_hire_assignments = []
    for v in pulp_prob.variables():
        if round(v.varValue) > 0:
            if 'x_soi' in v.name:
                model = v.name.split('_')[4]
                #change the task number to match with the instances
                task = str(int(v.name.split('_')[3])+1)
                task_time = instances[model]['task_times'][task]
                assignment = {'station': v.name.split('_')[2],'model':model  , 'task': task, 'task_times': task_time}
                task_assignments.append(assignment)
            elif 'b_wtsl' in v.name:
                model = sequences[int(v.name.split('_')[2])]['sequence'][int(v.name.split('_')[4])]
                labor = {'scenario':v.name.split('_')[2], 'stage':v.name.split('_')[3], 'station': v.name.split('_')[4], 'model':model, 'workers': int(v.name.split('_')[5]) }
                labor_assignments.append(labor)
            elif 'Y_w' in v.name:
                print(v.name.split('_')[2])
                labor_hire = {'scenario':v.name.split('_')[2], 'scenario_workers': int(v.value()) }
                labor_hire_assignments.append(labor_hire)
                print(v.name, v.varValue)

    #turns task_assignments into a dataframe
    task_assignments_df = pd.DataFrame(task_assignments)
    labor_assignments_df = pd.DataFrame(labor_assignments)
    labor_hire_df = pd.DataFrame(labor_hire_assignments)
    #concatenates the 'task' column in task_assignments_df if the 'station' and 'model' columns are the same
    task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()
    #print(task_assignments_df.head(20))
    labor_assignments_df['sequence_loc'] = labor_assignments_df['stage'].astype(int) - labor_assignments_df['station'].astype(int)
    labor_assignments_df = labor_assignments_df[labor_assignments_df['sequence_loc'] >= 0]
    task_seq = task_assignments_df[['station', 'model','task','task_times']]
    #merging labor and task sequence dataframes
    labor_task_seq = pd.merge(labor_assignments_df, task_seq, on=['model','station'], how='left')
    labor_task_seq = pd.merge(labor_task_seq, labor_hire_df, on=['scenario'], how='left')
    task_assignments_df.to_csv(file_name + f'task_assignment.csv', sep=' ')
    labor_task_seq.to_csv(file_name + f'labor_assignment.csv', sep=' ')
    return task_assignments_df, labor_assignments_df
#task_assignment, labor_assignment = generate_task_assignments_df_stochastic(prob, prod_sequences, test_instances)
#task_assignment

In [166]:
def pair_instances(instance_list, MODEL_MIXTURES):
      #Assuming we will not have more than 26 models
      model_names = list(string.ascii_uppercase)
      instance_groups = []
      for i in range(len(instance_list)-len(MODEL_MIXTURES)):
         instance_dict = {}
         for j in range(i, i+ len(MODEL_MIXTURES)):
            model_name = list(MODEL_MIXTURES.keys())[j-i]
            instance_dict[model_name] = {'fp':instance_list[j], 'name':model_name, 'probability':MODEL_MIXTURES[model_name]}
            
         instance_groups.append(instance_dict)
      return instance_groups

In [167]:
# instance_dicts = [
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_2.alb",'name':'B', 'probability':0.25}
# ]
def read_instance_folder(folder_loc):
   instance_list = []
   for file in glob.glob(f"{folder_loc}*.alb"):
      instance_list.append(file)
   instance_list.sort(key = lambda file: int(file.split("_")[-1].split(".")[0]))
   return instance_list

instance_list = read_instance_folder("SALBP_benchmark/small data set_n=20/")[:200]
# print(instance_list)
# instance_dicts = [
#    {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",'name':'B', 'probability':0.25}
# ]
# instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
#    "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",
#    ]
# instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
#    "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",
#    "SALBP_benchmark/small data set_n=20/instance_n=20_3.alb",]
#test_instances = create_instance_pair_stochastic(instance_dicts)


NO_EQUIPMENT = 4
seed = 1
NO_WORKERS =4
NO_STATIONS = 4
WORKER_COST = 500
TAKT_TIME = 1000
NO_TAKTS = 4
#MODEL_MIXTURES = {'A':0.50, 'B':0.40, 'C':0.10}
MODEL_MIXTURES = {'A':0.60, 'B':0.40}
#TODO Make this work for more than 2 models
def pair_instances(instance_list, MODEL_MIXTURES):
      '''returns a list of lists of multi-model instances, where each list of instances is a list of instances that will be run together'''
      instance_groups = []
      for i in range(len(instance_list)-len(MODEL_MIXTURES)+1):
         instance_group= []
         for j in range(i, i+ len(MODEL_MIXTURES)):
            model_name = list(MODEL_MIXTURES.keys())[j-i]
            instance_group.append({'fp':instance_list[j], 'name':model_name, 'probability':MODEL_MIXTURES[model_name]})
         instance_groups.append(instance_group)
      return instance_groups

def obj_val_dict(instance_groups, obj_value, solver_status, test_instance):
   obj_val_dict = {}
   for i, input_instance in enumerate(instance_groups):
      obj_val_dict['file_'+ str(i)] = input_instance['fp']
      model = list(test_instance.keys())[i]
      for key, value in test_instance[model].items():
         obj_val_dict[model+'_'+key] = value
   obj_val_dict['obj_value'] = obj_value
   return obj_val_dict

    
def multi_run(instance_list, milp_model,report_generator, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed, file_name = 'test'):
   instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
   print(instance_groups)
   test_results = []
   group_counter = 0
   test_instances = []
   instance_results = []
   for group in instance_groups:
      test_instance = create_instance_pair_stochastic(group)
      print('Running instances', group)
      print('\n test_instance', test_instance)
      test_instances.append(test_instance)
      #create equipment
      print('creating equipment')
      all_tasks = get_task_union(test_instance, *list(test_instance.keys()) )
      no_tasks = len(all_tasks)
      c_se, r_oe = generate_equipment(NO_EQUIPMENT, NO_STATIONS, all_tasks, seed = seed)
      equipment_instance = {1: {'equipment_prices': c_se, 'equipment_matrix': r_oe}}
      #create scenario tree
      print('generating scenario tree')
      scenario_tree_graph, final_sequences = make_scenario_tree(NO_TAKTS, MODEL_MIXTURES)
      print('defining problem')
      prob = milp_model(test_instance, equipment_instance[1],no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_TAKTS, final_sequences, worker_cost=WORKER_COST)
      print('solving problem')
      solver = plp.GUROBI_CMD(options=[('TimeLimit', 180), ('MIPGap', 0.01)])#
      prob.solve(solver=solver)
      print('writing results')
      print('objective value', prob.objective.value())
      result = obj_val_dict(group, prob.objective.value(),prob.status, test_instance)
      instance_results.append(result)
      print('test_instance', test_instance)
      out_name = file_name + str(group_counter)
      task_assignment, labor_assignment = report_generator(prob, final_sequences, test_instance, out_name)
      group_counter += 1

   instance_results = pd.DataFrame(instance_results)
   instance_results.to_csv(file_name + '_results.csv')   

xp_name = "stochastic_new"
#makes a folder of xp_name if it does not exist already

if not os.path.exists('model_runs/'+ xp_name):
   os.makedirs('model_runs/'+xp_name)
file_name = 'model_runs/'+xp_name+f'/Stochastic_{NO_STATIONS}_E{NO_EQUIPMENT}_L{NO_WORKERS}_T{NO_TAKTS}_C{TAKT_TIME}_LC{WORKER_COST}_'
#file_name = 'model_runs/13_26stochastic_'
multi_run(instance_list,stochastic_problem_linear_labour,generate_report_stochastic, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed, file_name = file_name)
file_name = 'model_runs/'+xp_name+f'/Model_dependent_{NO_STATIONS}_E{NO_EQUIPMENT}_L{NO_WORKERS}_T{NO_TAKTS}_C{TAKT_TIME}_LC{WORKER_COST}_'
multi_run(instance_list, model_dependent_eq_linear_labour,generate_report_md, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed, file_name=file_name)

[[{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_1.alb', 'name': 'A', 'probability': 0.6}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_2.alb', 'name': 'B', 'probability': 0.4}], [{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_2.alb', 'name': 'A', 'probability': 0.6}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_3.alb', 'name': 'B', 'probability': 0.4}], [{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_3.alb', 'name': 'A', 'probability': 0.6}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_4.alb', 'name': 'B', 'probability': 0.4}], [{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_4.alb', 'name': 'A', 'probability': 0.6}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_5.alb', 'name': 'B', 'probability': 0.4}], [{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_5.alb', 'name': 'A', 'probability': 0.6}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_6.alb', 'name': 'B', 'probabilit

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_34510/1936757272.py:35: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


solving problem
Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.03
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ebef17a4ae18457983f5721b5a5b374c-pulp.lp
Reading time = 0.02 seconds
Total_cost: 10960 rows, 7392 columns, 53488 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10960 rows, 7392 columns and 53488 nonzeros
Model fingerprint: 0x50818699
Variable types: 0 continuous, 7392 integer (7376 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5523 rows and 3974 columns
Presolve time: 0.07s
Presolved: 5437 rows

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_34510/1936757272.py:35: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


solving problem
Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.03
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/f5223702ee75474282b5ea526751f2f4-pulp.lp
Reading time = 0.02 seconds
Total_cost: 10928 rows, 7392 columns, 53232 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10928 rows, 7392 columns and 53232 nonzeros
Model fingerprint: 0x33a5e48e
Variable types: 0 continuous, 7392 integer (7376 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5482 rows and 3949 columns
Presolve time: 0.07s
Presolved: 5446 rows

...

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_34510/1936757272.py:75: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.03
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/6c0ad7cddad946a4b9067f6389a9ea0f-pulp.lp
Reading time = 0.01 seconds
Total_cost: 1051 rows, 2432 columns, 11896 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1051 rows, 2432 columns and 11896 nonzeros
Model fingerprint: 0x0195c6fc
Variable types: 0 continuous, 2432 integer (2416 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 822 rows and 2276 columns
Presolve time: 0.01s
Presolved: 229 rows, 156 columns, 928 n

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_34510/1936757272.py:75: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.03
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/8b5c03a7d34b4b19b1412eabbbd694a2-pulp.lp
Reading time = 0.01 seconds
Total_cost: 1051 rows, 2432 columns, 11896 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1051 rows, 2432 columns and 11896 nonzeros
Model fingerprint: 0x8b891cec
Variable types: 0 continuous, 2432 integer (2416 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 822 rows and 2276 columns
Presolve time: 0.01s
Presolved: 229 rows, 156 columns, 935 n

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_34510/1936757272.py:75: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


## Constructive Heuristic for model dependent case

In [168]:
#Tasks selection methods
def longest_processing_time(model, candidate_list, **kwargs):
    max_task_time = 0
    for candidate in candidate_list:
        if model['task_times'][candidate] > max_task_time:
            max_task_time = model['task_times'][candidate]
            selected_task = candidate
    return selected_task


#Methods for the construction heurisitc

def calculate_takt_time(model, NO_S, TAKT_TIME):
    """
    Calculate the takt time for the model dependent model
    """
    total_task_time = sum(model['task_times'].values())
    new_takt_time = max(total_task_time/NO_S, TAKT_TIME)
    return new_takt_time

def model_task_assignment(model, all_tasks, NO_S, TAKT_TIME, selection_method, **kwargs):
    '''This function assigns the tasks of a model '''
    print('model', model)
    x_so = np.zeros((NO_S,len(all_tasks)))
    new_takt_time = calculate_takt_time( model, NO_S, TAKT_TIME)
    prec_matrix = construct_precedence_matrix(model)
    number_of_predecessor = np.sum(prec_matrix, axis=0)
    for station in range(NO_S):
        s_total_assingments = 0
        while s_total_assingments < new_takt_time and np.any(number_of_predecessor != -1):
            candidate_list = []
            for task in model['task_times']:
                task_in = int(task)-1
                if number_of_predecessor[task_in] == 0:
                    candidate_list.append(task)
            selected_task = selection_method(model, candidate_list,  **kwargs)
            selected_task_in = int(selected_task)-1
            x_so[station, selected_task_in] = 1
            s_total_assingments += model['task_times'][selected_task]
            number_of_predecessor -= prec_matrix[selected_task_in]
            number_of_predecessor[selected_task_in] = -1
        #If all the elements in number of predecessor are -1, then all tasks are assigned
        if np.all(number_of_predecessor == -1):
            break
    return x_so

def constructive_heurisitc(instance, NO_S, TAKT_TIME, selection_method):
    """
    Constructive heuristic for the model dependent model
    """
    print('instance', instance)
    all_tasks = get_task_union(instance, 'A', 'B')
    print('all_tasks', all_tasks)
    assignments = []
    #heuristic asssigns tasks for each model in instance
    for model in instance:
        x_so = model_task_assignment( instance[model], all_tasks, NO_S, TAKT_TIME, selection_method)
        assignments.append(x_so)
    #combines assignments list into a single numpy array, where eac
    x_soi = np.stack(assignments, axis=-1)
    return x_soi



In [169]:
def run_constructive_heuristic(instance_list, NO_S, TAKT_TIME, MODEL_MIXTURES):
    """
    Runs the constructive heuristic for all instances in the instance list
    """
    heuristics = []
    instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
    for group in instance_groups:
        instance = create_instance_pair_stochastic(group)
        heuristics.append(constructive_heurisitc(instance, NO_S, TAKT_TIME, longest_processing_time))
    return heuristics

def solve_from_initial_task_asssingments(instance, NO_S, TAKT_TIME, task_assignments):
    """
    Solves the instance using the given task assignments
    """

instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
   "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",
   ]
NO_S = 4
MODEL_MIXTURES = {'A': 0.5, 'B': 0.5}
results_heuristic = run_constructive_heuristic(instance_list, NO_S, TAKT_TIME, MODEL_MIXTURES)


instance {'A': {'num_tasks': 20, 'cycle_time': 1000, 'order_strength': 0.268, 'task_times': {'1': 142, '2': 34, '3': 140, '4': 214, '5': 121, '6': 279, '7': 50, '8': 282, '9': 129, '10': 175, '11': 97, '12': 132, '13': 107, '14': 132, '15': 69, '16': 169, '17': 73, '18': 231, '19': 120, '20': 186}, 'precedence_relations': [['1', '6'], ['2', '7'], ['4', '8'], ['5', '9'], ['6', '10'], ['7', '11'], ['8', '12'], ['10', '13'], ['11', '13'], ['12', '14'], ['12', '15'], ['13', '16'], ['13', '17'], ['13', '18'], ['14', '20'], ['15', '19']], 'probability': 0.5}, 'B': {'num_tasks': 20, 'cycle_time': 1000, 'order_strength': 0.3, 'task_times': {'1': 58, '2': 224, '3': 20, '4': 150, '5': 410, '6': 117, '7': 262, '8': 94, '9': 213, '10': 118, '11': 191, '12': 74, '13': 60, '14': 117, '15': 124, '16': 103, '17': 178, '18': 188, '19': 107, '20': 53}, 'precedence_relations': [['1', '13'], ['2', '5'], ['4', '6'], ['5', '10'], ['5', '11'], ['5', '12'], ['6', '7'], ['6', '8'], ['6', '9'], ['7', '13'], ['8

In [170]:
results_heuristic[0].shape

(4, 20, 2)

In [171]:
def summit(*args):
    total = 0
    for arg in args:
        total += arg
    return total

summit(*[1,2,3])

6